In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    pairwise_distances_argmin_min
)

from tqdm.notebook import tqdm
from scipy.ndimage import gaussian_filter1d


In [2]:
# ==== 0) Parâmetros ====
ARQ_IN    = "Matrix_X_Typical_set.csv"        # saída do passo anterior (pivot pronto)
OUT_W     = "W_frames.csv"      # pesos por frame (frames × K)
OUT_H     = "H_padroes.csv"     # padrões (K × células)
OUT_TOP   = "NMF_W_TS.csv"
SEED      = 42
MAX_ITER  = 2000

# ==== 1) Ler matriz ====
X = pd.read_csv(ARQ_IN, index_col=0)
X = X.clip(lower=0).astype(float)


In [3]:
print(f"Dimensões da matriz X: {X.shape}")  # (n_frames, n_cells)


Dimensões da matriz X: (850, 1075)


In [4]:
# ==== 4) Optimização com Optuna (multi-objetivo: KL + Calinski) ====
import optuna
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import calinski_harabasz_score

def objective(trial):
    # pesquisar nº de componentes (fatores latentes)
    n_components = trial.suggest_int("n_components", 2, min(40, X.shape[1]))

    alpha_H = trial.suggest_float("alpha_H", 1e-5, 1e-2, log=True)
    alpha_W = trial.suggest_float("alpha_W", 1e-5, 1e-2, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)

    nmf = NMF(
        n_components=n_components,
        init="nndsvda",
        solver="mu",
        beta_loss="kullback-leibler",
        max_iter=MAX_ITER,
        random_state=SEED,
        alpha_H=alpha_H,
        alpha_W=alpha_W,
        l1_ratio=l1_ratio
    )

    W = nmf.fit_transform(X.values)
    H = nmf.components_
    X_hat = W @ H

    # ----------------
    # 1. Divergência KL ↓
    # ----------------
    mask = X.values > 0
    kl = np.sum(
        X.values[mask] * np.log(X.values[mask] / np.maximum(X_hat[mask], 1e-10))
        - X.values[mask] + X_hat[mask]
    )

    # ----------------
    # 2. Calinski-Harabasz ↑
    # ----------------
    scaler = StandardScaler()
    W_scaled = scaler.fit_transform(W)

    kmeans = KMeans(n_clusters=n_components, random_state=SEED)
    labels = kmeans.fit_predict(W_scaled)

    try:
        calinski = calinski_harabasz_score(W_scaled, labels)
    except Exception:
        calinski = 0

    return kl, calinski


# ==== Multi-Objective Study ====
study = optuna.create_study(
    directions=["minimize", "maximize"],  # KL ↓, Calinski ↑
    sampler=optuna.samplers.NSGAIISampler(seed=SEED)
)

study.optimize(objective, n_trials=100)

# ==== Resultados ====
print("Número de soluções no Pareto front:", len(study.best_trials))
print("\n➡️ Soluções ótimas (compromissos KL vs Calinski):")
for t in study.best_trials:
    vals = t.values
    print(f"KL={vals[0]:.2f}, Calinski={vals[1]:.2f}, Params={t.params}")

# ---- Selecionar 1 solução do Pareto front ----
chosen_trial = study.best_trials[0]
best_params = chosen_trial.params
print("\n➡️ Solução mais equilibrada escolhida automaticamente:")
print(f"KL={chosen_trial.values[0]:.2f}, Calinski={chosen_trial.values[1]:.2f}")
print(f"Parâmetros: {best_params}")


[I 2025-10-10 17:35:47,279] A new study created in memory with name: no-name-67a81be4-9711-4b94-bb80-25fc546b2f15
[I 2025-10-10 17:35:48,230] Trial 0 finished with values: [137180.11536980403, 132.9922553398635] and parameters: {'n_components': 16, 'alpha_H': 0.0071144760093434225, 'alpha_W': 0.001570297088405539, 'l1_ratio': 0.5986584841970366}.
[I 2025-10-10 17:35:48,863] Trial 1 finished with values: [156223.34395696846, 328.3689392130336] and parameters: {'n_components': 8, 'alpha_H': 2.9375384576328295e-05, 'alpha_W': 1.493656855461762e-05, 'l1_ratio': 0.8661761457749352}.
[I 2025-10-10 17:35:50,244] Trial 2 finished with values: [116797.90959439203, 75.98054789889007] and parameters: {'n_components': 25, 'alpha_H': 0.001331121608073689, 'alpha_W': 1.1527987128232396e-05, 'l1_ratio': 0.9699098521619943}.
[I 2025-10-10 17:35:51,482] Trial 3 finished with values: [105398.07143725127, 51.617983261892235] and parameters: {'n_components': 34, 'alpha_H': 4.335281794951564e-05, 'alpha_W'

Número de soluções no Pareto front: 44

➡️ Soluções ótimas (compromissos KL vs Calinski):
KL=137180.12, Calinski=132.99, Params={'n_components': 16, 'alpha_H': 0.0071144760093434225, 'alpha_W': 0.001570297088405539, 'l1_ratio': 0.5986584841970366}
KL=156223.34, Calinski=328.37, Params={'n_components': 8, 'alpha_H': 2.9375384576328295e-05, 'alpha_W': 1.493656855461762e-05, 'l1_ratio': 0.8661761457749352}
KL=116797.91, Calinski=75.98, Params={'n_components': 25, 'alpha_H': 0.001331121608073689, 'alpha_W': 1.1527987128232396e-05, 'l1_ratio': 0.9699098521619943}
KL=105398.07, Calinski=51.62, Params={'n_components': 34, 'alpha_H': 4.335281794951564e-05, 'alpha_W': 3.511356313970405e-05, 'l1_ratio': 0.18340450985343382}
KL=141294.44, Calinski=164.00, Params={'n_components': 13, 'alpha_H': 0.00037520558551242813, 'alpha_W': 0.00019762189340280086, 'l1_ratio': 0.2912291401980419}
KL=116921.13, Calinski=85.71, Params={'n_components': 25, 'alpha_H': 2.621087878265438e-05, 'alpha_W': 7.5237428845

In [5]:
nmf_best = NMF(
    n_components=best_params["n_components"],
    init="nndsvda",
    solver="mu",
    beta_loss="kullback-leibler",
    max_iter=MAX_ITER,
    random_state=SEED,
    alpha_H=best_params["alpha_H"],
    alpha_W=best_params["alpha_W"],
    l1_ratio=best_params["l1_ratio"]
)

W_best = nmf_best.fit_transform(X.values)
H_best = nmf_best.components_

# Guardar em DataFrames
df_W = pd.DataFrame(W_best, index=X.index, columns=[f"padrao_{i+1}" for i in range(W_best.shape[1])])
df_H = pd.DataFrame(H_best, columns=X.columns, index=[f"padrao_{i+1}" for i in range(H_best.shape[0])])

print(f"✅ df_W e df_H prontos com Optuna (K={best_params['n_components']})")


✅ df_W e df_H prontos com Optuna (K=16)


In [6]:
# ==== Reconstrução final ====
X_hat = W_best @ H_best

# ==== Métricas de erro (NMF final, KL) ====

# Divergência KL
mask = X.values > 0
kl_div = np.sum(
    X.values[mask] * np.log(X.values[mask] / np.maximum(X_hat[mask], 1e-10))
    - X.values[mask] + X_hat[mask]
)

# RMSE e MAE auxiliares
diff = X.values - X_hat
rmse = np.sqrt((diff ** 2).mean())
mae  = np.abs(diff).mean()

print(f"Divergência KL: {kl_div:.6f}")
print(f"RMSE: {rmse:.6f}, MAE: {mae:.6f}")


Divergência KL: 137180.115370
RMSE: 0.783586, MAE: 0.131546


In [7]:
df_W.head()

,padrao_1,padrao_2,padrao_3,padrao_4,padrao_5,padrao_6,padrao_7,padrao_8,padrao_9,padrao_10,padrao_11,padrao_12,padrao_13,padrao_14,padrao_15,padrao_16
timestamp,,,,,,,,,,,,,,,,
1900-01-01 00:00:00.852,0.000000,0.000000e+00,9.431295e-155,4.226021e-83,1.195915e-46,0.000000,4.593524e-101,4.508182e-78,2.806507e-59,1.863434e+00,1.012762e+00,1.009208e-81,1.281967e-36,3.629010e-117,1.750227e-20,0.00000
1900-01-01 00:00:03.401,0.262673,0.000000e+00,5.756467e-252,1.953594e-82,5.694325e-19,0.599497,2.788895e-51,2.802952e-20,3.734614e-61,1.502444e-220,5.382139e-69,0.000000e+00,2.958861e-01,2.446862e-78,1.687845e+00,0.00000
1900-01-01 00:00:07.974,0.000000,0.000000e+00,1.308255e+00,0.000000e+00,1.138317e-271,0.000000,4.752867e-57,3.591706e-76,0.000000e+00,2.669327e-01,4.382321e-115,0.000000e+00,1.575743e-82,1.414540e-143,0.000000e+00,0.00000
1900-01-01 00:00:08.614,0.000000,1.649873e-47,0.000000e+00,6.107327e-01,0.000000e+00,0.002216,0.000000e+00,1.455044e-79,2.497476e-01,0.000000e+00,0.000000e+00,4.036409e-01,2.688885e-23,2.562473e-01,0.000000e+00,0.74553
1900-01-01 00:00:09.458,0.000000,0.000000e+00,1.468883e+00,6.498513e-184,9.167106e-90,0.000000,1.319310e-48,0.000000e+00,0.000000e+00,5.520660e-58,0.000000e+00,1.458454e-129,0.000000e+00,0.000000e+00,3.110900e-14,0.00000


In [8]:
df_W.to_csv("nmf_W_TS.csv")
